In [8]:
import copy
import sys
import os
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms


sys.path.append(os.path.dirname(os.getcwd()) + "/quantization/models")
from AlexNet import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
class Pipeline(object):
    def __init__(self, config):
        self.train_path = config["train_path"]
        self.test_path = config["test_path"]
        self.num_epoch = config["num_epoch"]
        self.lr = config["lr"]
        self.input_size = config["input_size"]
        self.batch_size = config["batch_size"]
        
        self.train_loader = None
        self.test_loader = None
        self.device = None
        self.model = None
        self.optimizer = None
        self.criterion = None
    
    
    def load_data(self):
        train_transform = transforms.Compose([
            transforms.CenterCrop(self.input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        test_transform = transforms.Compose([
            transforms.CenterCrop(self.input_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        train_set = ImageFolder(self.train_path, transform=train_transform)
        test_set = ImageFolder(self.test_path, transform=test_transform)
        
        self.train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=self.batch_size, shuffle=True)
        self.test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=self.batch_size, shuffle=False)
        
        
    def show(self):
        pass
    
    
    def load_model(self, model):
        self.device = torch.device('cpu')
        
        self.model = model.to(self.device)

        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.criterion = nn.CrossEntropyLoss().to(self.device)
        
        
    def train(self):
        print("train:")
        self.model.train()
        train_loss = 0
        train_correct = 0
        total = 0
        
        for epoch in range(self.num_epoch):
            print('Epoch {}/{}'.format(epoch+1, self.num_epoch))
            print('-'*15)

            for batch_num, (data, label) in enumerate(self.train_loader):
                data, label = data.to(self.device), label.to(self.device)
            
                self.optimizer.zero_grad()
                output = self.model(data)
                loss = self.criterion(output, label)
                loss.backward()
                self.optimizer.step()
                
                train_loss += loss.item()
                prediction = torch.max(output, 1)
                total += label.size(0)
                train_correct += np.sum(prediction[1].cpu().numpy() == label.cpu().numpy())
                
                print(batch_num, len(self.train_loader), 'Loss: %.4f | Acc: %.3f%% (%d/%d)'
                        % (train_loss / (batch_num + 1), 100. * train_correct / total, train_correct, total))

        return train_loss, train_correct / total
    
    def test(self):
        print("test:")
        self.model.eval()
        test_loss = 0
        test_correct = 0
        total = 0

        with torch.no_grad():
            for batch_num, (data, label) in enumerate(self.test_loader):
                data, label = data.to(self.device), label.to(self.device)
                output = self.model(data)
                loss = self.criterion(output, label)
                test_loss += loss.item()
                prediction = torch.max(output, 1)
                total += label.size(0)
                test_correct += np.sum(prediction[1].cpu().numpy() == label.cpu().numpy())

                print(batch_num, len(self.test_loader), 'Loss: %.4f | Acc: %.3f%% (%d/%d)'
                             % (test_loss / (batch_num + 1), 100. * test_correct / total, test_correct, total))

        return test_loss, test_correct / total

In [37]:
pipeline_config = {
    "train_path": "/home/yurii.laba/Projects/research/datasets/waste-classification-data/dataset/DATASET/TRAIN",
    "test_path": "/home/yurii.laba/Projects/research/datasets/waste-classification-data/dataset/DATASET/TEST",
    "input_size": 227,
    "batch_size": 100,
    "num_epoch": 1,
    "lr": 0.001
}

In [38]:
alexnet = AlexNet(2)
pipeline = Pipeline(pipeline_config)
pipeline.load_data()
pipeline.load_model(alexnet)
pipeline.train()

train:
Epoch 1/1
---------------
0 226 Loss: 0.6855 | Acc: 62.000% (62/100)
1 226 Loss: 30.0310 | Acc: 61.000% (122/200)
2 226 Loss: 20.3053 | Acc: 54.667% (164/300)
3 226 Loss: 15.4219 | Acc: 52.750% (211/400)
4 226 Loss: 12.4783 | Acc: 49.600% (248/500)
5 226 Loss: 10.5159 | Acc: 47.667% (286/600)
6 226 Loss: 9.1136 | Acc: 47.429% (332/700)
7 226 Loss: 8.0580 | Acc: 47.875% (383/800)
8 226 Loss: 7.2335 | Acc: 49.444% (445/900)
9 226 Loss: 6.5856 | Acc: 49.200% (492/1000)
10 226 Loss: 6.0458 | Acc: 49.545% (545/1100)
11 226 Loss: 5.5971 | Acc: 50.667% (608/1200)
12 226 Loss: 5.2184 | Acc: 50.846% (661/1300)
13 226 Loss: 4.8903 | Acc: 51.786% (725/1400)
14 226 Loss: 4.5993 | Acc: 53.267% (799/1500)
15 226 Loss: 4.3496 | Acc: 54.188% (867/1600)
16 226 Loss: 4.1272 | Acc: 55.235% (939/1700)
17 226 Loss: 3.9300 | Acc: 56.111% (1010/1800)
18 226 Loss: 3.7525 | Acc: 57.105% (1085/1900)
19 226 Loss: 3.5944 | Acc: 57.750% (1155/2000)
20 226 Loss: 3.4499 | Acc: 58.381% (1226/2100)
21 226 Loss:

171 226 Loss: 0.8257 | Acc: 77.233% (13284/17200)
172 226 Loss: 0.8231 | Acc: 77.283% (13370/17300)
173 226 Loss: 0.8205 | Acc: 77.333% (13456/17400)
174 226 Loss: 0.8186 | Acc: 77.366% (13539/17500)
175 226 Loss: 0.8170 | Acc: 77.386% (13620/17600)
176 226 Loss: 0.8146 | Acc: 77.435% (13706/17700)
177 226 Loss: 0.8127 | Acc: 77.438% (13784/17800)
178 226 Loss: 0.8101 | Acc: 77.492% (13871/17900)
179 226 Loss: 0.8082 | Acc: 77.500% (13950/18000)
180 226 Loss: 0.8064 | Acc: 77.514% (14030/18100)
181 226 Loss: 0.8047 | Acc: 77.527% (14110/18200)
182 226 Loss: 0.8027 | Acc: 77.552% (14192/18300)
183 226 Loss: 0.8004 | Acc: 77.587% (14276/18400)
184 226 Loss: 0.7982 | Acc: 77.627% (14361/18500)
185 226 Loss: 0.7961 | Acc: 77.651% (14443/18600)
186 226 Loss: 0.7949 | Acc: 77.636% (14518/18700)
187 226 Loss: 0.7928 | Acc: 77.665% (14601/18800)
188 226 Loss: 0.7911 | Acc: 77.693% (14684/18900)
189 226 Loss: 0.7894 | Acc: 77.721% (14767/19000)
190 226 Loss: 0.7871 | Acc: 77.775% (14855/19100)


(164.3119662106037, 0.7861194823612835)

In [39]:
pipeline.test()

test:
0 26 Loss: 0.1334 | Acc: 94.000% (94/100)
1 26 Loss: 0.1483 | Acc: 95.000% (190/200)
2 26 Loss: 0.1369 | Acc: 96.333% (289/300)
3 26 Loss: 0.1211 | Acc: 97.000% (388/400)
4 26 Loss: 0.1199 | Acc: 97.200% (486/500)
5 26 Loss: 0.1280 | Acc: 96.167% (577/600)
6 26 Loss: 0.1283 | Acc: 96.143% (673/700)
7 26 Loss: 0.1267 | Acc: 96.500% (772/800)
8 26 Loss: 0.1333 | Acc: 96.333% (867/900)
9 26 Loss: 0.1386 | Acc: 95.800% (958/1000)
10 26 Loss: 0.1345 | Acc: 96.000% (1056/1100)
11 26 Loss: 0.1355 | Acc: 96.083% (1153/1200)
12 26 Loss: 0.1412 | Acc: 95.846% (1246/1300)
13 26 Loss: 0.1460 | Acc: 95.429% (1336/1400)
14 26 Loss: 0.1500 | Acc: 95.467% (1432/1500)
15 26 Loss: 0.1545 | Acc: 95.562% (1529/1600)
16 26 Loss: 0.1637 | Acc: 95.412% (1622/1700)
17 26 Loss: 0.1728 | Acc: 95.056% (1711/1800)
18 26 Loss: 0.2222 | Acc: 92.579% (1759/1900)
19 26 Loss: 0.2550 | Acc: 90.850% (1817/2000)
20 26 Loss: 0.2888 | Acc: 89.714% (1884/2100)
21 26 Loss: 0.3081 | Acc: 89.000% (1958/2200)
22 26 Loss: 

(8.898750744760036, 0.8822124950258655)